### Bird Math and Simulations

Alright so I love the game wingspan a lot and when I love something, I want to do calculations. So here it goes. Let's look into a few questions, some simple, some hard(ish):

1. How many possible choices can you make with your opening hands
2. What are the cdfs for arbitrary numbers food from the bird feeder
3. What is the distribution of "difficulty" of playing various birds?
4. How does #3 compare the point values?
5. Look at the bonus cards?


### Set up:

So I found [this excel sheet](https://boardgamegeek.com/filepage/193164/wingspan-spreadsheet-bird-cards-bonus-cards-end-ro) that has the birds in it. Let's start by loading it up and doing whatever formatting we think might help:

In [128]:
import pandas as pd
import numpy as np
from math import comb
from random import choices

bird_list = pd.read_excel("wingspan-card-lists-20220718.xlsx")

In [129]:
bird_list.head()

,Common name,Scientific name,Set,Color,Power text,Flavor text,Predator,Flocking,Bonus card,Victory points,...,Large Bird Specialist,Nest Box Builder,Omnivore Expert,Passerine Specialist,Platform Builder,Prairie Manager,Rodentologist,Viticulturalist,Wetland Scientist,Wildlife Gardener
0,Abbott's Booby,Papasula abbotti,oceania,white,"Draw 3 bonus cards, then discard 2. You may di...","This large, endangered seabird lives only on C...",NaN,NaN,X,5,...,X,NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN
1,Acorn Woodpecker,Melanerpes formicivorus,core,brown,"Gain 1 [seed] from the birdfeeder, if availabl...","In a single dead tree, these birds may drill a...",NaN,NaN,NaN,5,...,NaN,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,American Avocet,Recurvirostra americana,core,pink,"When another player takes the ""lay eggs"" actio...","American avocets build their own nests, but th...",NaN,NaN,NaN,6,...,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN
3,American Bittern,Botaurus lentiginosus,core,brown,Player(s) with the fewest birds in their [wetl...,"Bitterns hide in reeds and cattails, where the...",NaN,NaN,NaN,7,...,X,NaN,NaN,NaN,X,NaN,X,NaN,X,NaN
4,American Coot,Fulica americana,core,brown,Tuck 1 [card] from your hand behind this bird....,"These omnivorous birds eat a lot of algae, but...",NaN,X,NaN,3,...,NaN,NaN,X,NaN,X,NaN,NaN,NaN,X,NaN


Some edits I'm going to do:

* Rename a lot of the columns since I'm a snake case fanboy
* Turn the indicator columns into binaries
* Add zeros to count columns instead of missing
* Subset to only base set (murica!)

In [130]:
bird_list.rename(
    columns = {
        '/ (food cost)' : 'food_slash',
        'Wild (food)' : 'wild',
        '* (food cost)' : 'food_star'
    },
    inplace = True
)

bird_list.rename(
    mapper = lambda x: x.lower().replace(' ', '_'),
    axis = 1,
    inplace = True
)

indicator_cols = [
    'bonus_card', 
    'forest',
    'grassland',
    'wetland',
    'anatomist', 
    'cartographer', 
    'historian', 
    'photographer',
    'backyard_birder', 
    'bird_bander', 
    'bird_counter', 
    'bird_feeder',
    'diet_specialist', 
    'enclosure_builder', 
    'falconer', 
    'fishery_manager',
    'food_web_expert', 
    'forester', 
    'large_bird_specialist',
    'nest_box_builder', 
    'omnivore_expert', 
    'passerine_specialist',
    'platform_builder', 
    'prairie_manager', 
    'rodentologist',
    'viticulturalist', 
    'wetland_scientist', 
    'wildlife_gardener'
]
bird_list[indicator_cols] = bird_list[indicator_cols].apply(lambda x : x.isna().astype('int'))

count_cols = [
    'invertebrate', 
    'seed', 
    'fish', 
    'fruit', 
    'rodent', 
    'nectar',
    'wild', 
    'food_slash', 
    'food_star', 
    'total_food_cost'
]
bird_list[count_cols] = bird_list[count_cols].apply(lambda x: x.fillna(0)) 

bird_list = bird_list.query("set == 'core'").reset_index()

In [131]:
bird_list[['forest', 'grassland', 'wetland', 'invertebrate', 'seed', 'fish', 'rodent', 'fruit', 'wild', 'food_slash', 'total_food_cost']].describe()

,forest,grassland,wetland,invertebrate,seed,fish,rodent,fruit,wild,total_food_cost
count,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,0.511765,0.511765,0.500000,0.735294,0.582353,0.223529,0.200000,0.247059,0.200000,1.964706
std,0.501338,0.501338,0.501477,0.657628,0.743202,0.530189,0.539559,0.496279,0.493748,0.813307
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,1.000000,1.000000,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,1.000000,1.000000,1.000000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,3.000000


This isn't quite what I thought in terms of hte distributions. I thought there was less balance in the habitats than there is. I also thought that seeds were the most common and that there was a bigger gap between fruit and fish/rodent.

### Number of starting choices:

The question that got me started on this is actually the easiest by far which is just "how many choices" can you make when choosing your starting food, birds, * bonus cards. Very easy to compute:

$${10 \choose 5}*2 = 504$$

You're allowed to select any combination of 5 items from the 10 distinct you start with. Basically I just wanted to say that this is one of the worst aspects of the game. I highly dislike that the first few times you sit down it's like "alright, are you ready to try to figure out which of these 504 choices is the best one? Also this is probably the most important decision point of the game. Hope then next 90 minutes are fun."

When you play a bit more, the choices shrink rapidly as you can more easily eliminate birds (and thus food) from consideration but still, it's a lot of choices. It's also true that the bonus card doesn't play into my decision making that much other than if it can match what birds I pick. Maybe we'll change our mind with some analysis of that later

### Food distributions on rerolling the bird feeder

Next question I have is what the probability of getting N or greater of a particular kind of food from a re-roll of the birdfeeder are. This problem comes about a lot when you're hoping to get a particular food from re-rolling the bird feeder and don't know how likely it is. We can just consider seeds (p = 1/3) and fish (p = 1/6) since the other are going to be mirrors of these.

If $X$ is the number of hits rolled then for seeds the CDF is like:

$$P(X \geq N) = \sum_{i=N}^{5}{5 \choose i}\left(\frac{1}{3}\right)^{i}\left(\frac{2}{3}\right)^{5-i}$$

And if $Y$ is the number of fish:

$$P(Y \geq N) = \sum_{i=N}^{5}{5 \choose i}\left(\frac{1}{6}\right)^{i}\left(\frac{5}{6}\right)^{5-i}$$

Compute

In [132]:
def cdf(n,p):
    total = 0
    for i in range(n, 6):
        total += comb(5,i)*(p**i)*((1-p)**(5-i))
    return round(100*total,1)
        
pd.DataFrame({
    'N' : list(range(6)),
    'seeds' : map(lambda x: cdf(x, 1/3), range(6)),
    'fish' : map(lambda x: cdf(x, 1/6), range(6))
})

,N,seeds,fish
0,0,100.0,100.0
1,1,86.8,59.8
2,2,53.9,19.6
3,3,21.0,3.5
4,4,4.5,0.3
5,5,0.4,0.0


Pretty true to my experience. You can't quite count on getting a seed and you should definitely not count on getting a fish. This especially relevant to the bald eagle and northern flicker abilities. The eagle is mostly there to just be worth 9 points IMO since you'll rarely get more than one fish from it. Flicker is kinda interesting when compared to the american goldfinch or brown pelican. The finch and pelican give you a lot better payout than the flicker but are considerably harder to play (can't be done out of the opening hand). I don't really like the flicker that much and I think this curve shows why -- about half the time it only replaces it nets food only about half the time.

Let's double check this math with simulation. I also want to create a birdfeeder for the next question anyways:

In [162]:
food_list =  ['invertebrates', 'seeds', 'dual', 'fruit', 'rodents', 'fish']

class birdfeeder:
    def __init__(self):
        self.dice = dict(zip(food_list, [0 for i in range(6)]))
        self.re_roll()
    
    def re_roll(self):
        self.dice = dict(zip(food_list, [0 for i in range(6)]))
        new_food = choices(food_list, k = 5)
        for food in new_food:
            self.dice[food] += 1
            
    def eat(self, food):
        if self.dice[food] > 0:
            self.dice[food] -= 1
        else:
            raise ValueError

In [164]:
feeder = birdfeeder()
feeder.dice

{'invertebrates': 3,
 'seeds': 1,
 'dual': 0,
 'fruit': 1,
 'rodents': 0,
 'fish': 0}

In [161]:
# Now simulate:

